In [1]:
import numpy as np
from datasets import load_dataset
from transformers import AutoImageProcessor, AutoModel

tiny_imagenet = load_dataset('zh-plus/tiny-imagenet', split='valid')
processor = AutoImageProcessor.from_pretrained('facebook/dinov2-small')
model = AutoModel.from_pretrained('facebook/dinov2-small')

/home/czaloom/valor/.env-velour/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def embed(x):
    inputs = processor(images=x, return_tensors="pt")
    outputs = model(**inputs)
    retval = (
        outputs
        .last_hidden_state
        .flatten()
        .detach()
        .numpy()
    )
    retval = retval[np.newaxis, :]
    return retval

In [3]:
classes = {
    0: "fish",
    1: "ground_spider",
    2: "frog",
    5: "snake",
    8: "web_spider",
    19: "penguin"
}
id_labels = classes.keys()
text_labels = list(classes.values())

In [4]:
embeddings = [
    [
        embed(image['image'])
        for image in tiny_imagenet 
        if image['label'] == label
    ]
    for label in id_labels
]

Compute metrics

In [5]:
from embedding_metrics import compute_cvm, compute_ks, create_dataframe

In [6]:
cvm = compute_cvm(embeddings, text_labels)
ks = compute_ks(embeddings, text_labels)

Create pandas dataframes

In [7]:
cvm_statistics, cvm_pvalues = create_dataframe(cvm, text_labels)
ks_statistics, ks_pvalues = create_dataframe(ks, text_labels)

In [8]:
cvm_statistics

Reference                                             \
                         fish ground_spider     frog    snake web_spider   
Query fish              0.332       300.961  238.778  316.986    328.888   
      ground_spider   326.160         0.893  101.768  181.940    250.241   
      frog            222.880        47.346    0.125  170.924    255.216   
      snake           197.857        17.905   43.895    1.097    191.244   
      web_spider      342.481       262.132  295.753  328.617      2.745   
      penguin         214.925       202.980  202.526  232.556    262.217   

                              
                     penguin  
Query fish           324.187  
      ground_spider  345.564  
      frog           323.082  
      snake          238.504  
      web_spider     360.070  
      penguin          7.855

In [9]:
cvm_pvalues

Reference                                               
                         fish ground_spider   frog  snake web_spider penguin
Query fish               0.11         0.000  0.000  0.000        0.0     0.0
      ground_spider      0.00         0.004  0.000  0.000        0.0     0.0
      frog               0.00         0.000  0.475  0.000        0.0     0.0
      snake              0.00         0.000  0.000  0.001        0.0     0.0
      web_spider         0.00         0.000  0.000  0.000        0.0     0.0
      penguin            0.00         0.000  0.000  0.000        0.0     0.0

In [10]:
ks_statistics

Reference                                               
                         fish ground_spider   frog  snake web_spider penguin
Query fish              0.051         0.737  0.660  0.773      0.786   0.784
      ground_spider     0.779         0.065  0.404  0.534      0.659   0.816
      frog              0.626         0.258  0.033  0.521      0.669   0.774
      snake             0.577         0.166  0.272  0.073      0.571   0.643
      web_spider        0.831         0.689  0.740  0.798      0.095   0.866
      penguin           0.634         0.601  0.608  0.648      0.686   0.168

In [11]:
ks_pvalues

Reference                                               
                         fish ground_spider   frog  snake web_spider penguin
Query fish              0.087         0.000  0.000  0.000        0.0     0.0
      ground_spider     0.000         0.011  0.000  0.000        0.0     0.0
      frog              0.000         0.000  0.531  0.000        0.0     0.0
      snake             0.000         0.000  0.000  0.003        0.0     0.0
      web_spider        0.000         0.000  0.000  0.000        0.0     0.0
      penguin           0.000         0.000  0.000  0.000        0.0     0.0